<a href="https://colab.research.google.com/github/JeonChaeHwan/oss/blob/main/osmnx3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [ ]:
import networks as nx
import osmnx as ox
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd
from descartes import PolygonPatch
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon

In [ ]:
# configure the place, network type, trip times, and travel speed
address = "Hadan-dong, Saha-gu, Busan, Korea"
network_type = "walk"
trip_times = [1, 5, 10, 15, 20, 25, 30]
travel_speed = 4.5
G = ox.graph_from_address( address, network_type = network_type)

In [ ]:
# 위치에 따른 지도 가져오기
G = ox.graph_from_address(address, network_type=network_type)

# 원하는 지점과 해당 지도에 대해서 그래프
gdf_nodes = ox.graph_topdfs(G, edges=False)
# x, y = gdf_nodes["geometry"].unary_union.centroid.xy   # 지도 중심점 좌표
# 동아대학교의 위경도 좌표: lat -> 위도 x, 경도 -> lon y
x = 128.96817249950897
y = 36.11755694483541
center_node = ox.distance.nearest_nodes(G, x, y)
G = ox.project_graph(G)

그래프에 가중치 넣기

In [ ]:
from ast import keyword
# 계산을 위한 edges에 속성값을 넣음
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in G.edges(data=True, keys=True):
    data["time"] = data["length"] * meters_per_minute

등시성(isochrone) 맵 구현

In [ ]:
# 등시성의 polygon 구현
isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
  subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
  node_points = [Point((data["x"], data["y"])) for node, data in subgraph.node(data=True)]
  bounding_poly = gpd.GeoSeries(node_points).unaray_union.conves_hull
  isochrone_polys.append(bounding_poly)

# 등시성에 따른 네트워크를 표기
fig, ax = ox.plot_graph(
    G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0)
for polygon, fc in zip(isochrone_polys, iso_colors):
  patch = PolygonPatch(polygon, fc=fc, ec="none", alpha=0.6, zorder=1)
  ax.add_patch(patch)
plt.show()

Polyline 기반 등시성(isochrone) 맵 구현

In [ ]:
def make_iso_polys(G, edge_buff=25, node_buff=50, infill-False):
  isochrone_polys = []
  for trip_time in sorted(trip_times, reverse=True):
    subgraph=nx.ego_graph(G, center_node, radius=trip_time, distance="time")

    node_points = [Point((data["x"], data["y"])) for node, data in subgraph.node(data=True)]
    nodes_gdf = gpd.GeoDataFrame({"id": subgraph.nodes(), "geometry": node_points})
    nodes_gdf = nodes_gdf.set_index("id")

    edge_lines = []
    for n_fr, n_to in subgraph.edges() :
      f = nodes_gdf.loc[n_fr].geometry
      t = nodes_gdf.loc[n_to].geometry
      edge_lookup = G.get_edgs_data(n_fr, n_to[0].get("geometry", LineString([f, t])))
      edge_lines.append(edge_lookup)

    n = nodes_gdf.buffer(node_buff).geometry
    e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
    all_gs = list(n) + list(e)
    new_iso = gpd.GeoSeries(all_gs).unary_union

    if infill:
      new_iso = Polygon(new_iso.exterior)
    isochrone_polys.append(new_iso)
  return isochrone_polys

Isolated Polyline 기반 등시성(isochrone) 맵 구현

In [ ]:
isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
fig, ax = ox.plot_graph(
    G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0)
for polygon, fc in zip(isochrone_polys, iso_colors):
  patch = PolygonPatch(polygon, fc=fc, ec="none", alpha=0.6, zorder=1)
  ax.add_patch(patch)
plt.show()